# Group 1 Project--Anthony Sanchez

- **Kernel:** SQL (Azure Data Studio recommended)  
- **DB:** Northwinds2022TSQLV7  


### Database confirmation

In [ ]:
SELECT
  DB_NAME()     AS [current_database],
  SUSER_SNAME() AS [server_login],
  USER_NAME()   AS [db_user];


## Case 1 — Empty Orders

**Case Briefing:** Some orders exist without any line items which are suspicious 'empty shells'. We must locate orders with zero details and rank them by recency.

**Task 1:** Pull minimal fields for all orders so we have the base population.

In [ ]:
-- task1: orders
SELECT o.OrderID, o.CustomerID, o.OrderDate
INTO #c1_task1
FROM Sales.[Order] o;


**Task 2:** Count how many detail rows each order has.

In [ ]:
-- task2: detail counts
SELECT d.OrderID, COUNT(*) AS line_count
INTO #c1_task2
FROM Sales.OrderDetail d
GROUP BY d.OrderID;


**Task 3:** Join counts back onto orders to get a per-order line tally.

In [ ]:
-- task3: join counts
SELECT t1.OrderID, t1.CustomerID, t1.OrderDate,
       ISNULL(t2.line_count, 0) AS line_count
INTO #c1_task3
FROM #c1_task1 t1
LEFT JOIN #c1_task2 t2 ON t2.OrderID = t1.OrderID;


**Task 4:** Filter to orders with **zero** line items (potential data issues).

In [ ]:
-- task4: keep empties 
SELECT *
INTO #c1_task4
FROM #c1_task3
WHERE line_count = 0;


**Task 5:** Rank empty orders by recency and display results.

In [ ]:
-- task5: rank & final select 
SELECT *, ROW_NUMBER() OVER (ORDER BY OrderDate DESC, OrderID DESC) AS recency_rank
INTO #c1_task5
FROM #c1_task4;

SELECT OrderID, CustomerID, OrderDate, line_count, recency_rank
FROM #c1_task5
ORDER BY recency_rank;


**Clear once found**

In [ ]:
-- Cleanup for Case 1
DROP TABLE IF EXISTS #c1_task1;
DROP TABLE IF EXISTS #c1_task2;
DROP TABLE IF EXISTS #c1_task3;
DROP TABLE IF EXISTS #c1_task4;
DROP TABLE IF EXISTS #c1_task5;


## Case 2 — Charged Double

**Case Briefing:** There are potential double charges as the same customer places multiple orders on the same day with the same total. We must compute totals, group by day and customer, and surface duplicates.

**Task 1 Briefing:** Compute value at the line level so discounts are accounted for.

In [ ]:
-- task1: line net
SELECT d.OrderID,
       CAST(d.UnitPrice AS decimal(18,4)) * d.Quantity * (1 - ISNULL(d.Discount,0)) AS line_net
INTO #c2_task1
FROM Sales.OrderDetail d;


**Task 2 Briefing:** Aggregate line nets into per-order totals.

In [ ]:
-- task2: order totals
SELECT OrderID, CAST(ROUND(SUM(line_net), 2) AS decimal(19,2)) AS order_total
INTO #c2_task2
FROM #c2_task1
GROUP BY OrderID;


**Task 3 Briefing:** Attach the customer and the order date.

In [ ]:
-- task3: attach customer + day
SELECT o.OrderID, o.CustomerID, CAST(o.OrderDate AS date) AS order_day
INTO #c2_task3
FROM Sales.[Order] o;


**Task 4 Briefing:** Find groups of (customer, day, total) with 2+ orders.

In [ ]:
-- task4: dupe groups
SELECT t3.CustomerID, t3.order_day, t2.order_total, COUNT(*) AS order_count
INTO #c2_task4
FROM #c2_task3 t3
JOIN #c2_task2 t2 ON t2.OrderID = t3.OrderID
GROUP BY t3.CustomerID, t3.order_day, t2.order_total
HAVING COUNT(*) >= 2;


**Task 5 Briefing:** List all orders belonging to those duplicate groups.

In [ ]:
-- task5: list members & final 
SELECT t3.CustomerID, t3.order_day, t2.order_total, t3.OrderID
INTO #c2_task5
FROM #c2_task4 g
JOIN #c2_task3 t3 ON t3.CustomerID = g.CustomerID AND t3.order_day = g.order_day
JOIN #c2_task2 t2 ON t2.OrderID = t3.OrderID AND t2.order_total = g.order_total;

SELECT CustomerID, order_day, order_total, OrderID
FROM #c2_task5
ORDER BY order_day DESC, order_total DESC, CustomerID, OrderID;


**Clear once found**

In [ ]:
-- Cleanup for Case 2
DROP TABLE IF EXISTS #c2_task1;
DROP TABLE IF EXISTS #c2_task2;
DROP TABLE IF EXISTS #c2_task3;
DROP TABLE IF EXISTS #c2_task4;
DROP TABLE IF EXISTS #c2_task5;


## Case 4 — Too Good to Be True

**Case Briefing:** There are very risky discounts going around with very large discount percentages on high-value lines. We must compute value then rank the most suspicious.

**Task 1:** Collect raw order line fields and normalize types for math.

In [ ]:
-- task1: raw lines 
SELECT d.OrderID, d.ProductID,
       CAST(d.UnitPrice AS decimal(18,4)) AS UnitPrice,
       d.Quantity,
       ISNULL(d.Discount,0) AS DiscountPct
INTO #c4_task1
FROM Sales.OrderDetail d;


**Task 2:** Compute gross, discount value, and carry discount percentage.

In [ ]:
-- task2: value math 
SELECT OrderID, ProductID,
       UnitPrice * Quantity AS gross_value,
       UnitPrice * Quantity * DiscountPct AS discount_value,
       DiscountPct
INTO #c4_task2
FROM #c4_task1;


**Task 3:** Apply business thresholds.

In [ ]:
-- task3: thresholds 
SELECT *
INTO #c4_task3
FROM #c4_task2
WHERE DiscountPct >= 0.40  -- 40%+
  AND gross_value >= 200;  -- big-ticket


**Task 4:** Compute net and discount ratio for sorting.

In [ ]:
-- task4: compute net & ratio 
SELECT *, (gross_value - discount_value) AS net_value,
       CASE WHEN gross_value > 0 THEN discount_value / gross_value END AS discount_ratio
INTO #c4_task4
FROM #c4_task3;


**Task 5:** Rank suspicious lines and show the top entries.

In [ ]:
-- task5: rank & final 
SELECT *, ROW_NUMBER() OVER (ORDER BY discount_value DESC, discount_ratio DESC) AS suspicion_rank
INTO #c4_task5
FROM #c4_task4;

SELECT TOP (50) OrderID, ProductID, gross_value, discount_value, discount_ratio, net_value, suspicion_rank
FROM #c4_task5
ORDER BY suspicion_rank;


**Clear once found**

In [ ]:
-- Cleanup for Case 4
DROP TABLE IF EXISTS #c4_task1;
DROP TABLE IF EXISTS #c4_task2;
DROP TABLE IF EXISTS #c4_task3;
DROP TABLE IF EXISTS #c4_task4;
DROP TABLE IF EXISTS #c4_task5;


## Case 5 — Employees with Zero Orders

**Case Briefing:** Identify employees who never appear on orders—potential onboarding, access, or tracking gap.

**Task 1:** List all employees.

In [ ]:
-- task1: all employees
SELECT e.EmployeeID
INTO #c5_task1
FROM HumanResources.Employee e;


**Task 2:** Collect the set of employees that appear on any order.

In [ ]:
-- task2: employees seen on orders
SELECT DISTINCT ISNULL(o.EmployeeID, -1) AS EmployeeID
INTO #c5_task2
FROM Sales.[Order] o;


**Task 3:** Anti-join to find employees not present in orders.

In [ ]:
-- task3: anti-join
SELECT t1.EmployeeID
INTO #c5_task3
FROM #c5_task1 t1
LEFT JOIN #c5_task2 t2 ON t2.EmployeeID = t1.EmployeeID
WHERE t2.EmployeeID IS NULL;


**Task 4:** Double-check order counts for those employees.

In [ ]:
-- task4: confirm counts
SELECT t3.EmployeeID, COUNT(o.OrderID) AS order_count
INTO #c5_task4
FROM #c5_task3 t3
LEFT JOIN Sales.[Order] o ON o.EmployeeID = t3.EmployeeID
GROUP BY t3.EmployeeID;


**Task 5:** Keep employees with **zero** orders and output.

In [ ]:
-- task5: keep truly invisible & final
SELECT EmployeeID
INTO #c5_task5
FROM #c5_task4
WHERE order_count = 0;

SELECT EmployeeID
FROM #c5_task5
ORDER BY EmployeeID;


**Clear once found**

In [ ]:
-- Cleanup for Case 5
DROP TABLE IF EXISTS #c5_task1;
DROP TABLE IF EXISTS #c5_task2;
DROP TABLE IF EXISTS #c5_task3;
DROP TABLE IF EXISTS #c5_task4;
DROP TABLE IF EXISTS #c5_task5;


## Case 7 — Days Gone Missing

**Case Briefing:** Find dates with zero orders within the last 180 days—useful for spotting outages, holidays, or data gaps.

**Task 1:** Construct a 180-day calendar.

In [ ]:
-- task1: last 180-day calendar
SELECT CAST(DATEADD(day, n, DATEADD(day, -179, CAST(GETDATE() AS date))) AS date) AS d
INTO #c7_task1
FROM (SELECT TOP (180) ROW_NUMBER() OVER (ORDER BY (SELECT NULL)) - 1 AS n FROM dbo.Nums) x;


**Task 2:** Count orders per calendar date (date-only grain).

In [ ]:
-- task2: orders per day
SELECT CAST(o.OrderDate AS date) AS d, COUNT(*) AS orders_count
INTO #c7_task2
FROM Sales.[Order] o
WHERE o.OrderDate >= DATEADD(day, -179, CAST(GETDATE() AS date))
GROUP BY CAST(o.OrderDate AS date);


**Task 3:** Left-join counts to the full calendar to surface missing days.

In [ ]:
-- task3: join calendar
SELECT t1.d, ISNULL(t2.orders_count,0) AS orders_count
INTO #c7_task3
FROM #c7_task1 t1
LEFT JOIN #c7_task2 t2 ON t2.d = t1.d;


**Task 4:** Filter to days with zero orders.

In [ ]:
-- task4: detect gaps
SELECT d
INTO #c7_task4
FROM #c7_task3
WHERE orders_count = 0;


**Task 5:** Rank gaps by recency and output.

In [ ]:
-- task5: rank & final
SELECT d, ROW_NUMBER() OVER (ORDER BY d DESC) AS gap_rank
INTO #c7_task5
FROM #c7_task4;

SELECT d AS MissingOrderDate, gap_rank
FROM #c7_task5
ORDER BY d DESC;


**Clear once found**

In [ ]:
-- Cleanup for Case 7
DROP TABLE IF EXISTS #c7_task1;
DROP TABLE IF EXISTS #c7_task2;
DROP TABLE IF EXISTS #c7_task3;
DROP TABLE IF EXISTS #c7_task4;
DROP TABLE IF EXISTS #c7_task5;
